# Scraping APIs

A site with an API (Application Programming Interface) wants you to scrape it.

Examples abound:

* <a href="https://www.census.gov/data/developers/data-sets.html">U.S. Census APIs</a>
* <a href="https://apps.fas.usda.gov/opendataweb/home">US Agriculture Commodities and Exports</a>
* <a href="https://www.federalregister.gov/developers/documentation/api/v1">Federal Register</a>
* <a href="https://developer.dol.gov/beginners-guide/">Labor Department</a>
* <a href="https://www.eia.gov/">Labor Department</a>

Government sites tend  ```CSVs``` for download but their APIs offer more detailed options for data. They are not trying to hide the data.

Private sites might have APIs, but often charge heafty prices for access beyond a basic number of downloads.

The toughest/hardest part of scraping an API is that they ***ALL HAVE DIFFERENT INSTRUCTIONS*** on how to tap their data.

Today, we'll explore different APIs that each build a different skill:

1. Census health data – **building a simple API call.**
2. USDA commodities exports – **using an API key and targeting specific commodities over several years.**
3. Federal Register – **tapping search terms.**
4. Energy Information Administration – **dealing with pagination.**

What they all have in common:

1. a base url
2. a query string
3. tied together with a query character ```?```
4. an API key.

Combined together these are known as an ```API endpoint```.

You make an ```API call``` (a request) using the ```API endpoint```.




In [1]:
pip install icecream

Note: you may need to restart the kernel to use updated packages.


In [2]:
## import libraries
import requests
import pandas as pd
from icecream import ic



### 1. Census health data – **building a simple API call.**

- <a href="https://www.census.gov/data/developers/data-sets/Health-Insurance-Statistics.html">Census health landing page</a>
- List of <a href="https://api.census.gov/data/timeseries/healthins/sahie/variables.html">possible variables</a>

We want to create a dataframe with the following info for every state in 2021:

1. Total number insured
2. Percent insured
3. Total number uninsured
4. Percent uninsured
5. Broken down by race

## the parts to build your API endpoint.

In [3]:
## create a dictionary to know what codes mean
target_dict = {
    "NIC_PT": "total_insured",
    "PCTIC_PT": "pct_insured_est",
    "NUI_PT": "total_uninsured",
    "PCTUI_PT": "pct_uninsured_est",
    "RACECAT": "race_category",
    "RACE_DESC": "race_description",
    "STABREV": "state"
    
}

In [4]:
## keys() to get keys
list(target_dict.keys())

['NIC_PT', 'PCTIC_PT', 'NUI_PT', 'PCTUI_PT', 'RACECAT', 'RACE_DESC', 'STABREV']

In [5]:
## values() to get val;;ues
list(target_dict.values())

['total_insured',
 'pct_insured_est',
 'total_uninsured',
 'pct_uninsured_est',
 'race_category',
 'race_description',
 'state']

In [6]:
## format into api query format
my_vars = ",".join(target_dict.keys())
my_vars

'NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV'

In [7]:
## endpoint
base_url = "https://api.census.gov/data/timeseries/healthins/sahie?get="
target_year = "2021"

In [8]:
## create query string
query_str = f"{my_vars}&for=state:*&time={target_year}"
query_str

'NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV&for=state:*&time=2021'

In [9]:
## create full API call
endpoint = base_url + query_str
endpoint

'https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV&for=state:*&time=2021'

In [10]:
## get response
response = requests.get(endpoint)



In [11]:
## turn response into json
data = response.json()
data

[['NIC_PT',
  'PCTIC_PT',
  'NUI_PT',
  'PCTUI_PT',
  'RACECAT',
  'RACE_DESC',
  'STABREV',
  'time',
  'state'],
 ['3548525', '88.3', '469887', '11.7', '0', 'All Races', 'AL', '2021', '01'],
 ['2293718',
  '90.0',
  '254385',
  '10.0',
  '1',
  'White alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['921007',
  '87.1',
  '135939',
  '12.9',
  '2',
  'Black or African American alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['163769',
  '73.3',
  '59615',
  '26.7',
  '3',
  'Hispanic or Latino (any race)',
  'AL',
  '2021',
  '01'],
 ['13274',
  '83.7',
  '2587',
  '16.3',
  '4',
  'American Indian and Alaska Native alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['57789',
  '89.8',
  '6551',
  '10.2',
  '5',
  'Asian alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['1784',
  '84.9',
  '317',
  '15.1',
  '6',
  'Native Hawaiian and Other Pacific Islander alone, not Hispanic or Latino',
  'AL',
  '2021',
  '01'],
 ['97184',
  '90.3',
  '1

In [12]:
## create dataframe
df = pd.DataFrame(data[1:], columns =data[0])
df

,NIC_PT,PCTIC_PT,NUI_PT,PCTUI_PT,RACECAT,RACE_DESC,STABREV,time,state
0,3548525,88.3,469887,11.7,0,All Races,AL,2021,01
1,2293718,90.0,254385,10.0,1,"White alone, not Hispanic or Latino",AL,2021,01
2,921007,87.1,135939,12.9,2,"Black or African American alone, not Hispanic ...",AL,2021,01
3,163769,73.3,59615,26.7,3,Hispanic or Latino (any race),AL,2021,01
4,13274,83.7,2587,16.3,4,"American Indian and Alaska Native alone, not H...",AL,2021,01
...,...,...,...,...,...,...,...,...,...
403,39116,71.4,15696,28.6,3,Hispanic or Latino (any race),WY,2021,56
404,5559,70.8,2290,29.2,4,"American Indian and Alaska Native alone, not H...",WY,2021,56
405,4033,83.5,795,16.5,5,"Asian alone, not Hispanic or Latino",WY,2021,56
406,248,79.0,66,21.0,6,Native Hawaiian and Other Pacific Islander alo...,WY,2021,56


In [13]:
## rename column headers with more meaning full headers
df.rename(columns = target_dict, inplace = True)
df

,total_insured,pct_insured_est,total_uninsured,pct_uninsured_est,race_category,race_description,state,time,state
0,3548525,88.3,469887,11.7,0,All Races,AL,2021,01
1,2293718,90.0,254385,10.0,1,"White alone, not Hispanic or Latino",AL,2021,01
2,921007,87.1,135939,12.9,2,"Black or African American alone, not Hispanic ...",AL,2021,01
3,163769,73.3,59615,26.7,3,Hispanic or Latino (any race),AL,2021,01
4,13274,83.7,2587,16.3,4,"American Indian and Alaska Native alone, not H...",AL,2021,01
...,...,...,...,...,...,...,...,...,...
403,39116,71.4,15696,28.6,3,Hispanic or Latino (any race),WY,2021,56
404,5559,70.8,2290,29.2,4,"American Indian and Alaska Native alone, not H...",WY,2021,56
405,4033,83.5,795,16.5,5,"Asian alone, not Hispanic or Latino",WY,2021,56
406,248,79.0,66,21.0,6,Native Hawaiian and Other Pacific Islander alo...,WY,2021,56


### 2. USDA commodities exports – **using an API key and targeting specific commodities over several years.**

- <a href="https://apps.fas.usda.gov/opendataweb/home">USDA APIs endpoints</a>
- Get an <a href="https://apps.fas.usda.gov/opendataweb/home">API key</a>

We want to create a dataframe with exports between 2020-2022 to all countries for the following commodities:

1. All Wheat
2. Oats
3. Cuts of Beef
4. Cuts of Pork

In [14]:
## find the parts to build your API call

In [15]:
## get commodities list
com_url = "https://apps.fas.usda.gov/OpenData/api/esr/commodities"

In [16]:
## we create a headers with your API key- use your own
headers = {
    'API_KEY': 'ADD-YOUR-KEY'
}

In [17]:
## now let's put into get requests
## we check the response status code
response  = requests.get(url = com_url, headers = poop)
response.status_code
all_commodities = response.json()
all_commodities

NameError: name 'poop' is not defined

### Get endpoint and test out on a single commodity


In [ ]:
## your end point here
## trying on all wheat 

endpoint = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/107/allCountries/marketYear/2020"
# countries = "https://apps.fas.usda.gov/OpenData/api/esr/countries"

In [ ]:
## now let's put into get requests
## we check the response status code
response = requests.get(url = endpoint, headers = headers)
response.status_code

In [ ]:
## let's store our response into an object called data
response.json()

In [ ]:
## convert that list of dicts into a dataframe called df
df = pd.DataFrame(response.json())
df

In [ ]:
## Now iterate through all our target items

my_commodities = [
    {'commodityCode': 107, 'commodityName': 'All Wheat', 'unitId': 1},
    {'commodityCode': 601, 'commodityName': 'Oats', 'unitId': 1},
    {'commodityCode': 1701, 'commodityName': 'Beef',
  'unitId': 1},
    {'commodityCode': 1702, 'commodityName': 'Pork',
  'unitId': 1}]

my_commodities

In [ ]:
my_commodities[0].get('commodityCode')

In [ ]:
## endpoint templates
start_endpoint = "https://apps.fas.usda.gov/OpenData/api/esr/exports/commodityCode/"
end_endpoint = "/allCountries/marketYear/"

In [ ]:
## iterate to get all the data
target_data = []
failed_endpoints = []
for commodity in my_commodities:
    target_code = commodity.get('commodityCode')
    ic(target_code)
    for year in range(2020,2023):
        ic(year)
        try:
            endpoint = f"{start_endpoint}{target_code}{end_endpoint}{year}"
            ##get response
            response = requests.get(url = endpoint, headers = headers)
            ## turn into dataframe
#             df = pd.DataFrame(response.json)
            target_data.append(pd.DataFrame(response.json()))
        except:
            print(f"Failed to retrieve data from {endpoint}...got status code {response.status_code}")
            failed_endpoints.append(endpoint)
print("done scraping API")    

In [ ]:
## call list
len(target_data)

In [ ]:
## concat into single df
df = pd.concat(target_data).reset_index(drop = True)

In [ ]:
## call df
df

In [ ]:
## confirm we have all our target commodities
list(df["commodityCode"].unique())

### 3. Federal Register – **tapping search terms.**

We have decades of <a href="https://docs.google.com/spreadsheets/d/130WeumbMZjcoRP4D-1uJ7bM0aKBZzt4N/edit?usp=sharing&ouid=112307892189798608417&rtpof=true&sd=true">SBA Excel files</a> that detail loans given to small businesses to recover after climate disasters. The only information we have about the type of disasters are codes in one of the columns that look like:

- CA-00279
- IL-00051
- NC-00099
- CA-00288
- LA-00079

The <a href="https://www.federalregister.gov/">Federal Register</a> allows us to search for what these codes stand for. But we can't search for nearly a thousand such disaster codes. When we try to scrape the site, it warns us to use the API instead.

Federal Register <a href="https://www.federalregister.gov/developers/documentation/api/v1#/Federal%20Register%20Documents/get_documents__format_">API documentation</a>

In [ ]:
## find the end point



#### Test on single endpoint after figuring out how to build API call

In [20]:
## endpoint
endpoint = "https://www.federalregister.gov/api/v1/documents.json?per_page=20&conditions[docket_id]=CA-00279"

In [21]:
## get data
response = requests.get(endpoint)
data = response.json()
data

{'count': 5,
 'description': 'Documents filed under agency docket CA-00279',
 'total_pages': 1,
 'results': [{'title': 'Presidential Declaration Amendment of a Major Disaster for the State of California',
   'type': 'Notice',
   'abstract': 'This is an amendment of the Presidential declaration of a major disaster for the State of California (FEMA-4344-DR), dated 10/12/ 2017. Incident: Wildfires. Incident Period: 10/08/2017 through 10/31/2017.',
   'document_number': '2017-24850',
   'html_url': 'https://www.federalregister.gov/documents/2017/11/16/2017-24850/presidential-declaration-amendment-of-a-major-disaster-for-the-state-of-california',
   'pdf_url': 'https://www.govinfo.gov/content/pkg/FR-2017-11-16/pdf/2017-24850.pdf',
   'public_inspection_pdf_url': 'https://public-inspection.federalregister.gov/2017-24850.pdf?1510753574',
   'publication_date': '2017-11-16',
   'agencies': [{'raw_name': 'SMALL BUSINESS ADMINISTRATION',
     'name': 'Small Business Administration',
     'id': 4

In [22]:
## type
type(data)

dict

In [27]:
## targeting incidents
data.get("results")[0].get("abstract")

'This is an amendment of the Presidential declaration of a major disaster for the State of California (FEMA-4344-DR), dated 10/12/ 2017. Incident: Wildfires. Incident Period: 10/08/2017 through 10/31/2017.'

### Iterate through entire list of codes

In [28]:
## Normally will take from df as a list
## build disaster code list
disaster_codes = ["CA-00279","IL-00051", "NC-00099", "CA-00288", "LA-00079" ]


In [29]:
## provide base url
base_url = "https://www.federalregister.gov/api/v1/documents.json?per_page=20&conditions[docket_id]="

In [37]:
## iterate through all endpoints
incidents_list = []
broken_endpoints = []

for disaster_code in disaster_codes:
#     print(disaster_code)
    endpoint = base_url + disaster_code
#     print(endpoint)
    try:
        response = requests.get(endpoint)
        data = response.json()
        incident_text = data.get("results")[0].get("abstract")
#         print(incident_text)
        incidents_list.append({"disaster_code": disaster_code, "incident_text": incident_text})
    except:
        pass
incidents_list

[{'disaster_code': 'CA-00279',
  'incident_text': 'This is an amendment of the Presidential declaration of a major disaster for the State of California (FEMA-4344-DR), dated 10/12/ 2017. Incident: Wildfires. Incident Period: 10/08/2017 through 10/31/2017.'},
 {'disaster_code': 'IL-00051',
  'incident_text': 'This is a notice of an Administrative declaration of a disaster for the State of Illinois dated 05/07/2018. Incident: Severe Storms and Flooding. Incident Period: 02/14/2018 through 03/04/2018.'},
 {'disaster_code': 'NC-00099',
  'incident_text': 'This is an amendment of the Presidential declaration of a major disaster for the State of North Carolina (FEMA-4393-DR), dated 09/14/2018. Incident: Hurricane Florence. Incident Period: 09/07/2018 through 09/29/2018.'},
 {'disaster_code': 'CA-00288',
  'incident_text': 'This is an amendment of the Presidential declaration of a major disaster for the State of California (FEMA-4382-DR), dated 08/04/ 2018. Incident: Wildfires and High Winds.

In [42]:
## call list
# Write to CSV file
# Extract the relevant information
rows = [(entry['disaster_code'], entry['incident_text'].split('Incident:')[1].split('Incident Period:')[0].strip()) for entry in incidents_list]

# Create a DataFrame
df = pd.DataFrame(rows, columns=['disaster_code', 'incident_text'])

# Write to CSV file using df.to_csv
csv_filename = 'disaster_data_pandas.csv'
df.to_csv(csv_filename, index=False)

print(f'CSV file "{csv_filename}" created successfully.')

CSV file "disaster_data_pandas.csv" created successfully.


### 4. Energy Information Administration – **dealing with pagination.**

From the <a href="https://www.eia.gov/">Energy Information Administration</a>, we want to compile electricity generation by type of fuel and region for about 5 days hourly.

We will encounter a limit on the number of items per API call.

Find our API endpoint first.

In [44]:
## your target endpoint
endpoint = "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?frequency=hourly&data[0]=value&start=2023-01-01T00&end=2023-01-07T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=Bo53FXqDefHvjNY97PmF3XYdfOap8hOaced0YFeN"

In [45]:
## get response
response = requests.get(endpoint)
data = response.json()
data

{'response': {'warnings': [{'warning': 'incomplete return',
    'description': 'The API can only return 5000 rows in JSON format.  Please consider constraining your request with facet, start, or end, or using offset to paginate results.'}],
  'total': 56260,
  'dateFormat': 'YYYY-MM-DD"T"HH24',
  'frequency': 'hourly',
  'data': [{'period': '2023-01-07T00',
    'respondent': 'NYIS',
    'respondent-name': 'New York Independent System Operator',
    'fueltype': 'NUC',
    'type-name': 'Nuclear',
    'value': 3266,
    'value-units': 'megawatthours'},
   {'period': '2023-01-07T00',
    'respondent': 'SE',
    'respondent-name': 'Southeast',
    'fueltype': 'OIL',
    'type-name': 'Petroleum',
    'value': 0,
    'value-units': 'megawatthours'},
   {'period': '2023-01-07T00',
    'respondent': 'SOCO',
    'respondent-name': 'Southern Company Services, Inc. - Trans',
    'fueltype': 'WND',
    'type-name': 'Wind',
    'value': 0,
    'value-units': 'megawatthours'},
   {'period': '2023-01-

In [46]:
## import ceiling division from math

from math import ceil

In [60]:
## your target endpoint
endpoint = "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?frequency=hourly&data[0]=value&start=2023-01-01T00&end=2023-01-07T00&sort[0][column]=period&sort[0][direction]=desc&offset="
api_key= "&length=5000&api_key=Bo53FXqDefHvjNY97PmF3XYdfOap8hOaced0YFeN"

In [49]:
## total pages:
rows_per_page = 5000
total_rows = data.get("response").get("total")
total_pages = ceil(total_rows/rows_per_page)
total_pages

12

In [62]:
## paginate our API calls
## for loop with pagination

all_data = []

for page in range(total_pages):
#     print(page)
    offset = page * rows_per_page
    api_url = f"{endpoint}{offset}{api_key}"
    ic(api_url)
    response = requests.get(api_url)
    data = response.json()
    all_data.append(data.get("response").get("data"))

ic| api_url: 'https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?frequency=hourly&data[0]=value&start=2023-01-01T00&end=2023-01-07T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=Bo53FXqDefHvjNY97PmF3XYdfOap8hOaced0YFeN'
ic| api_url: 'https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?frequency=hourly&data[0]=value&start=2023-01-01T00&end=2023-01-07T00&sort[0][column]=period&sort[0][direction]=desc&offset=5000&length=5000&api_key=Bo53FXqDefHvjNY97PmF3XYdfOap8hOaced0YFeN'
ic| api_url: 'https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?frequency=hourly&data[0]=value&start=2023-01-01T00&end=2023-01-07T00&sort[0][column]=period&sort[0][direction]=desc&offset=10000&length=5000&api_key=Bo53FXqDefHvjNY97PmF3XYdfOap8hOaced0YFeN'
ic| api_url: 'https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/?frequency=hourly&data[0]=value&start=2023-01-01T00&end=2023-01-07T00&sort[0][column]=period&sort[0][direction]=desc&offset=15000&length=

In [63]:
## call all data
all_data

[[{'period': '2023-01-07T00',
   'respondent': 'ERCO',
   'respondent-name': 'Electric Reliability Council of Texas, Inc.',
   'fueltype': 'WND',
   'type-name': 'Wind',
   'value': 17873,
   'value-units': 'megawatthours'},
  {'period': '2023-01-07T00',
   'respondent': 'CPLW',
   'respondent-name': 'Duke Energy Progress West',
   'fueltype': 'COL',
   'type-name': 'Coal',
   'value': 0,
   'value-units': 'megawatthours'},
  {'period': '2023-01-07T00',
   'respondent': 'AECI',
   'respondent-name': 'Associated Electric Cooperative, Inc.',
   'fueltype': 'WND',
   'type-name': 'Wind',
   'value': 261,
   'value-units': 'megawatthours'},
  {'period': '2023-01-07T00',
   'respondent': 'CISO',
   'respondent-name': 'California Independent System Operator',
   'fueltype': 'WAT',
   'type-name': 'Hydro',
   'value': 2394,
   'value-units': 'megawatthours'},
  {'period': '2023-01-07T00',
   'respondent': 'ISNE',
   'respondent-name': 'ISO New England',
   'fueltype': 'NG',
   'type-name': 'N

In [64]:
## length
len(all_data)

12

In [57]:
## use itertools to flatten list with nested lists
import itertools

In [65]:
## flatten nested lists
flat_data = list(itertools.chain(*all_data))

In [66]:
## convert to df
df = pd.DataFrame(flat_data)
df

,period,respondent,respondent-name,fueltype,type-name,value,value-units
0,2023-01-07T00,ERCO,"Electric Reliability Council of Texas, Inc.",WND,Wind,17873.0,megawatthours
1,2023-01-07T00,CPLW,Duke Energy Progress West,COL,Coal,0.0,megawatthours
2,2023-01-07T00,AECI,"Associated Electric Cooperative, Inc.",WND,Wind,261.0,megawatthours
3,2023-01-07T00,CISO,California Independent System Operator,WAT,Hydro,2394.0,megawatthours
4,2023-01-07T00,ISNE,ISO New England,NG,Natural gas,8095.0,megawatthours
...,...,...,...,...,...,...,...
56255,2023-01-01T00,PACE,PacifiCorp East,NG,Natural gas,1529.0,megawatthours
56256,2023-01-01T00,NY,New York,SUN,Solar,0.0,megawatthours
56257,2023-01-01T00,FMPP,Florida Municipal Power Pool,NG,Natural gas,1202.0,megawatthours
56258,2023-01-01T00,NW,Northwest,OTH,Other,1004.0,megawatthours


In [ ]:
fips_codes = {
    'Alabama': '01',
    'Alaska': '02',
    'Arizona': '04',
    'Arkansas': '05',
    'California': '06',
    'Colorado': '08',
    'Connecticut': '09',
    'Delaware': '10',
    'Florida': '12',
    'Georgia': '13',
    'Hawaii': '15',
    'Idaho': '16',
    'Illinois': '17',
    'Indiana': '18',
    'Iowa': '19',
    'Kansas': '20',
    'Kentucky': '21',
    'Louisiana': '22',
    'Maine': '23',
    'Maryland': '24',
    'Massachusetts': '25',
    'Michigan': '26',
    'Minnesota': '27',
    'Mississippi': '28',
    'Missouri': '29',
    'Montana': '30',
    'Nebraska': '31',
    'Nevada': '32',
    'New Hampshire': '33',
    'New Jersey': '34',
    'New Mexico': '35',
    'New York': '36',
    'North Carolina': '37',
    'North Dakota': '38',
    'Ohio': '39',
    'Oklahoma': '40',
    'Oregon': '41',
    'Pennsylvania': '42',
    'Rhode Island': '44',
    'South Carolina': '45',
    'South Dakota': '46',
    'Tennessee': '47',
    'Texas': '48',
    'Utah': '49',
    'Vermont': '50',
    'Virginia': '51',
    'Washington': '53',
    'West Virginia': '54',
    'Wisconsin': '55',
    'Wyoming': '56',
    'District of Columbia': '11',
    'Puerto Rico': '72'
}
